In [2]:
import os
import pandas as pd

In [16]:
import os
import pandas as pd
import numpy as np

data_dir = "../data"
output_dir = "../outputs/match_qa"
os.makedirs(output_dir, exist_ok=True)

for match_folder in os.listdir(data_dir):
    match_path = os.path.join(data_dir, match_folder)
    if not os.path.isdir(match_path):
        continue

    # Find the Excel file in the folder
    excel_files = [f for f in os.listdir(match_path) if f.endswith(".xlsx")]
    if not excel_files:
        print(f"No Excel file found in {match_path}")
        continue

    file_path = os.path.join(match_path, excel_files[0])
    print(f"Processing: {file_path}")

    try:
        xls = pd.ExcelFile(file_path)

        # Parse sheets
        df_players = xls.parse("Sheet1")
        df_teams = xls.parse("Sheet2")

        # Clean player headers
        df_players.columns = df_players.iloc[0]
        print(df_players.columns, 'columns')
        df_players = df_players[1:].reset_index(drop=True)
        df_players = df_players.dropna(axis=1, how='all')

        # Convert columns safely
        for col in ['Gls', 'Cmp', 'SCA']:
            df_players[col] = pd.to_numeric(df_players[col], errors='coerce')

        # Extract player stats
        top_scorer = df_players.sort_values("Gls", ascending=False).iloc[0]
        most_passes = df_players.sort_values("Cmp", ascending=False).iloc[0]
        most_sca = df_players.sort_values("SCA", ascending=False).iloc[0]

        # Try to extract possession (look for float values between 0 and 1)
        possession = None
        for idx, row in df_teams.iterrows():
            possible_values = row.dropna().values
            float_vals = [v for v in possible_values if isinstance(v, (float, int)) and 0 <= v <= 1]
            if len(float_vals) >= 2:
                possession = float_vals
                break

        # Try to extract goals (look for row with strings that look like player names)
        goals_row = []
        for idx, row in df_teams.iterrows():
            text_values = row.dropna().astype(str).values
            if any(name for name in text_values if name.strip().isalpha() and len(name.split()) >= 1):
                goals_row = text_values
                break

        # Match label for Q&A
        match_name = match_folder.replace("_", " ").replace("-", "–")

        # Build Q&A
        qa_list = [
            {
                "question": f"Who scored the most goals for {match_name}?",
                "answer": f"{top_scorer['Player']} scored {int(top_scorer['Gls'])} goal(s)."
            },
            {
                "question": f"Which player completed the most passes in {match_name}?",
                "answer": f"{most_passes['Player']} completed {int(most_passes['Cmp'])} passes."
            },
            {
                "question": f"Who had the most shot-creating actions in {match_name}?",
                "answer": f"{most_sca['Player']} had {int(most_sca['SCA'])} shot-creating actions."
            },
        ]

        if possession:
            qa_list.append({
                "question": f"What was the possession in {match_name}?",
                "answer": f"Argentina had {possession[0]*100:.0f}% possession."
            })

        if goals_row.any():
            qa_list.append({
                "question": f"Who scored in {match_name}?",
                "answer": f"The goal scorers were: {', '.join(goals_row)}."
            })

        # Save Q&A to CSV
        out_path = os.path.join(output_dir, f"{match_folder}.csv")
        pd.DataFrame(qa_list).to_csv(out_path, index=False)
        print(f"Saved: {out_path}")

    except Exception as e:
        print(f"Failed to process {file_path}: {e}")


Processing: ../data\1_Arg_vs_Saudi_G1\Arg_vs_Saudi.xlsx
Index(['Player', '#', 'Pos', 'Age', 'Club', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG',
       'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att', 'Succ'],
      dtype='object', name=0) columns
Saved: ../outputs/match_qa\1_Arg_vs_Saudi_G1.csv
Processing: ../data\2_Arg_vs_Mex_G2\Arg_v- Mex.xlsx
Index(['Player', '#', 'Pos', 'Age', 'Club', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG',
       'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att', 'Succ'],
      dtype='object', name=0) columns
Saved: ../outputs/match_qa\2_Arg_vs_Mex_G2.csv
Processing: ../data\3_Arg_vs_Pol_G3\Arg_vs_Pol.xlsx
Index(['Player', '#', 'Pos', 'Age', 'Club', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int